In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool

In [2]:
SEED = 42

In [3]:
def load_obj(name: str) -> object:
    with open(name, "rb") as f:
        obj = pickle.load(f)
    return obj

In [4]:
data = pd.read_pickle('./data/proc_data.pickle')

#### данные содержат уже токенизированные тексты

In [5]:
data.head()  

,title,topic,target
0,британцы отмечают двухлетие смерти дианы,Мир,0
1,еще одно землетрясение турции человек погиб ок...,Мир,0
2,российские национал-большевики убирают террито...,Мир,0
3,киргизия ведет бои границах таджикистаном узбе...,Мир,0
4,литва засудила участников переворота 91 года,Мир,0


In [6]:
file_name = './data/lemm_titles.pkl'

#### lemm_titles включают лемматизированные заголовки

In [7]:
lemm_titles = load_obj(file_name)

In [8]:
targets = data.target  # категории текстов

In [9]:
targets.name = 'Category'

In [10]:
del data  # удалим df для экономии оперативной памяти

In [11]:
values = list(lemm_titles.values())
keys = list(lemm_titles.keys())

In [12]:
df = pd.DataFrame({'titles': values})

In [13]:
df = pd.concat([df, targets], axis=1)

In [14]:
df.head()

,titles,Category
0,британец отмечать двухлетие смерть диана,0
1,ещё один землетрясение турция человек погибнут...,0
2,российский национал-большевик убирать территор...,0
3,киргизия вести бой граница таджикистан узбекистан,0
4,литва засудить участник переворот 91 год,0


In [27]:
lem_titles = df.titles.to_list()

### tfidf Векторизация

In [61]:
vectorizer = TfidfVectorizer(max_df=0.85, min_df=0.003)
vectors = vectorizer.fit_transform(lem_titles)
selected_w = vectorizer.get_feature_names_out()
tfidf_features = vectors.toarray()

In [62]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_features, targets, test_size=0.2, random_state=SEED)

#### В качестве бейзлайна применяется KNN. Алгоритм вычисляет расстояние признакового описания текущего объекта и всех остальных объектов. Далее выбирается k ближайших объектов и текущему объекту присваивается таргет на основе голосования (классификация) или среднеее (регрессия).

In [63]:
knn = KNeighborsClassifier(n_neighbors=3)

In [64]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [65]:
y_pred = knn.predict(X_test)

### Так как эта задача мультиклассовой классификации, распределение категорий в целом схоже, мало представленных категорий документов нет, __accuracy__ является адекватной метрикой, позволит увидеть что алгоритм способен классифицировать все категории документов


In [66]:
accuracy_score(y_test, y_pred)

0.5733871714179933

### В качестве улучшения будет использоваться модель градиентного бустинга - композиционной модели из решающих деревьев, которая в отличие от knn действительно 'обучается', каждая базовая модель в композиции минимизирует энтропию

In [23]:
train_dataset = Pool(data=X_train, label=y_train)
valid_dataset = Pool(data=X_test, label=y_test)

In [25]:
clf = CatBoostClassifier(random_seed=SEED, task_type='GPU', devices='0', loss_function='MultiClass')

In [26]:
clf.fit(train_dataset, verbose=False, eval_set=valid_dataset, early_stopping_rounds=50)

In [27]:
y_pred = clf.predict(X_test)

In [28]:
accuracy_score(y_test, y_pred)

0.6324740836727138

##### В качестве улучшения признаков будут использоваться эмбединги языковой модели, учитывающей глобальный контекст слова в предложении

#### Использование предобученной на русских текстах модели  для получения эмбедингов каждого токена в заголовке

In [67]:
!pip install -q navec

In [71]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2023-08-21 15:12:59--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  16.4MB/s    in 3.4s    

2023-08-21 15:13:03 (15.1 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [72]:
from navec import Navec

In [73]:
path = './navec_hudlit_v1_12B_500K_300d_100q.tar'

In [74]:
navec = Navec.load(path)

In [75]:
def mean_embed_text(lem_titles):
    X, y = [], []
    
    for i, item in enumerate(tqdm(lem_titles)):
        embed_sent = []  # массив куда будут помещаться эмбединги каждого токена в текущем заголовке
        
        sent = item.split(' ')
        for word in sent:
            if word in navec: # если токен есть в модели
                embed = navec[word]
                embed_sent.append(embed)
        
        embed_sent = np.array(embed_sent)
        
        if embed_sent.sum() != 0:  # может быть случай, что в текущем заголовке не нашлось ни одного токена в модели
            embed_sent = embed_sent.mean(axis=0).reshape(1, -1)  # эмбединг заголовка получается усреднением каждого токена
            X.append(embed_sent)
            y.append(targets[i])
            
    X = np.concatenate(X)
    y = np.array(y)
    return X, y

In [76]:
X, y = mean_embed_text(lem_titles)
        
    

100%|██████████| 432158/432158 [00:54<00:00, 7860.19it/s]


In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

#### KNN

In [77]:
knn = KNeighborsClassifier(n_neighbors=3)

In [78]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [79]:
y_pred = knn.predict(X_test)

In [80]:
accuracy_score(y_test, y_pred)

0.5733871714179933

#### Catboost

In [92]:
train_dataset = Pool(data=X_train, label=y_train)
valid_dataset = Pool(data=X_test, label=y_test)

In [93]:
clf = CatBoostClassifier(random_seed=SEED, task_type='GPU', devices='0', loss_function='MultiClass')

In [94]:
clf.fit(train_dataset, verbose=200, eval_set=valid_dataset, early_stopping_rounds=50)

Learning rate set to 0.187234
0:	learn: 1.6131076	test: 1.6142960	best: 1.6142960 (0)	total: 84.7ms	remaining: 1m 24s
200:	learn: 0.5857572	test: 0.6205834	best: 0.6205834 (200)	total: 8.47s	remaining: 33.7s
400:	learn: 0.5104456	test: 0.5755603	best: 0.5755603 (400)	total: 16s	remaining: 24s
600:	learn: 0.4652047	test: 0.5555307	best: 0.5555307 (600)	total: 23.7s	remaining: 15.7s
800:	learn: 0.4314134	test: 0.5434316	best: 0.5434316 (800)	total: 31.3s	remaining: 7.77s
999:	learn: 0.4033543	test: 0.5348500	best: 0.5348500 (999)	total: 38.2s	remaining: 0us
bestTest = 0.5348499749
bestIteration = 999


In [95]:
y_pred = clf.predict(X_test)

In [96]:
accuracy_score(y_test, y_pred)

0.81874349184311

#### Попробую диверсифировать модель за счет валидации, каждая модель будет обучаться на отдельной подвыборке -> это делает ее более обобщаемой на разные данные

##### StratifiedKFold - стратегия, так чтобы и в обучении и в валидации было одинаковое распределение классов

In [112]:
from sklearn.model_selection import StratifiedKFold

In [116]:
skf = StratifiedKFold(n_splits=5)

In [124]:
clfs = []  # массив с обученными моделями
scores = [] # значение метрики на каждом разбиении

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    
    train_x, valid_x = X_train[train_index, :], X_train[test_index, :]
    train_y, valid_y = y_train[train_index], y_train[test_index]
    
    train_dataset = Pool(data=train_x, label=train_y)
    valid_dataset = Pool(data=valid_x, label=valid_y)
    
    clf = CatBoostClassifier(random_seed=SEED, iterations=3000, task_type='GPU', devices='0', loss_function='MultiClass')
    clf.fit(train_dataset, verbose=200, eval_set=valid_dataset, early_stopping_rounds=100)
    y_pred = clf.predict(valid_x)
    clfs.append(clf)
    
    score = accuracy_score(valid_y, y_pred)
    scores.append(score)
    

Learning rate set to 0.113265
0:	learn: 1.6785449	test: 1.6792979	best: 1.6792979 (0)	total: 71.7ms	remaining: 3m 35s
200:	learn: 0.6379960	test: 0.6621832	best: 0.6621832 (200)	total: 7.29s	remaining: 1m 41s
400:	learn: 0.5545690	test: 0.6037919	best: 0.6037919 (400)	total: 13.8s	remaining: 1m 29s
600:	learn: 0.5091109	test: 0.5790420	best: 0.5790420 (600)	total: 20s	remaining: 1m 19s
800:	learn: 0.4759340	test: 0.5635228	best: 0.5635228 (800)	total: 27s	remaining: 1m 14s
1000:	learn: 0.4489356	test: 0.5528584	best: 0.5528584 (1000)	total: 33s	remaining: 1m 5s
1200:	learn: 0.4257447	test: 0.5447321	best: 0.5447321 (1200)	total: 39.1s	remaining: 58.6s
1400:	learn: 0.4043756	test: 0.5377471	best: 0.5377471 (1400)	total: 45.2s	remaining: 51.6s
1600:	learn: 0.3856997	test: 0.5321511	best: 0.5321511 (1600)	total: 51.3s	remaining: 44.8s
1800:	learn: 0.3674698	test: 0.5269136	best: 0.5269136 (1800)	total: 58.3s	remaining: 38.8s
2000:	learn: 0.3514805	test: 0.5229248	best: 0.5229248 (2000)	to

In [126]:
print(*scores)  # значение метрик на валидации имеют примерно одинаковые значения

0.8258375894613312 0.8292920778168956 0.8250938001024777 0.828 0.8280330578512397


In [182]:
all_pred = []
for clf in clfs:
    y_pred = clf.predict(X_test)  # предсказание каждого классификатора
    all_pred.append(y_pred)
    
all_pred = np.array(all_pred).squeeze(2)
    

In [15]:
def avg_class(arr: np.ndarray) -> int:
    '''
    Вытаскиваем самый частотный предсказываемый класс среди пяти моделей
    '''
    unique_elements, counts = np.unique(arr, return_counts=True)
    max_count_index = np.argmax(counts)
    max_count = counts[max_count_index]
    most_frequent_elements = unique_elements[counts == max_count]
    
    return most_frequent_elements[0]  # если таких два, вытаскиваем первый
        

In [199]:
avg_pred = []
for i in range(all_pred.shape[1]):
    pred = all_pred[:, i]
    avg_pred.append(avg_class(pred))
    
avg_pred = np.array(avg_pred)
    

In [204]:
accuracy_score(y_test, avg_pred)

0.832858961008909

#### Также попробовал сделать взвешенную сумму эмбедингов токенов в соотвествии с TF-IDF, но это дало худшие результаты